In [8]:
! pip install datasets
! pip install huggingface_hub
! pip install pandas
! pip install soundfile
! pip install librosa
! pip install openpyxl


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
     ------------------------------------- 250.9/250.9 kB 16.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
# Select CUDA device index
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## Load Full Dataset (Not Recommended)

In [2]:
from datasets import load_dataset, Dataset
import random
import os

dataset_name = "recursal/reprocessed_singapore_national_speech_corpus"
custom_cache_dir = "../raw"  
dataset = load_dataset(dataset_name, cache_dir=custom_cache_dir)
# print(dataset)

Resolving data files:   0%|          | 0/98 [00:00<?, ?it/s]

## Stream & Load samples by intervals

In [12]:
from datasets import load_dataset
import os
import shutil
import pandas as pd
import json
import soundfile as sf
import numpy as np

dataset_name = "recursal/reprocessed_singapore_national_speech_corpus"
local_save_dir = "../raw"

# Create the directory if it doesn't exist
os.makedirs(local_save_dir, exist_ok=True)

# Load the dataset in streaming mode
dataset = load_dataset(dataset_name, streaming=True)

# Number of examples to collect per split
num_examples = 10

# Sampling interval - take every Nth example
interval = 50

# Process each split
for split_name, split_dataset in dataset.items():
    # Create a subdirectory for this split
    split_dir = os.path.join(local_save_dir, split_name)
    os.makedirs(split_dir, exist_ok=True)
    
    # Create a list to store all example data
    examples_data = []
    
    # Take samples at regular intervals
    for i, example in enumerate(split_dataset):
        # Only sample at the specified interval
        if i % interval != 0:
            continue
            
        # Process the audio file
        flac_data = example['flac']
        
        # Check the format of the flac data
        if isinstance(flac_data, dict) and 'array' in flac_data and 'path' in flac_data:
            # Extract the array and sampling rate
            audio_array = flac_data['array']
            sampling_rate = flac_data.get('sampling_rate', 16000)
            
            # Create a filename based on the original or use a sequential number
            filename = os.path.basename(flac_data['path']) if 'path' in flac_data else f"{split_name}_{len(examples_data)}.flac"
            audio_path = os.path.join(split_dir, filename)
            
            # Save the audio array to a file
            sf.write(audio_path, audio_array, sampling_rate)
        else:
            # If it's already a path, copy the file
            audio_path = flac_data
            filename = os.path.basename(audio_path)
            destination_path = os.path.join(split_dir, filename)
            shutil.copy(audio_path, destination_path)
            audio_path = destination_path
        
        # Create a dictionary with all the example data
        example_dict = dict(example)
        
        # Replace the flac data with the path to the saved file
        example_dict['flac'] = audio_path
        
        # Log which example we're processing
        print(f"Added example {i} (sample {len(examples_data)+1}/{num_examples})")
        
        # Add to our list of examples
        examples_data.append(example_dict)
        
        # Stop after collecting enough examples
        if len(examples_data) >= num_examples:
            break
    
    # Convert to DataFrame
    df = pd.DataFrame(examples_data)
    
    # Save as Excel (excluding large binary data if needed)
    excel_path = os.path.join(local_save_dir, f"{split_name}_data.xlsx")
    export_df = df.copy()
    for col in export_df.columns:
        # Convert any complex columns to string representation
        if export_df[col].dtype == 'object':
            export_df[col] = export_df[col].apply(lambda x: str(x) if not isinstance(x, (str, int, float, bool, type(None))) else x)
    export_df.to_excel(excel_path, index=False)
    
    # Save as CSV
    csv_path = os.path.join(local_save_dir, f"{split_name}_data.csv")
    export_df.to_csv(csv_path, index=False)
    
    # Save as JSON
    json_path = os.path.join(local_save_dir, f"{split_name}_data.json")
    
    # Handle complex objects for JSON serialization
    json_data = []
    for example in examples_data:
        json_example = {}
        for k, v in example.items():
            # Skip numpy arrays and other complex objects
            if isinstance(v, (str, int, float, bool, type(None))):
                json_example[k] = v
            elif isinstance(v, dict):
                # Convert sub-dictionaries properly
                json_example[k] = {str(sk): str(sv) if not isinstance(sv, (str, int, float, bool, type(None))) else sv 
                                 for sk, sv in v.items()}
            else:
                json_example[k] = str(v)
        json_data.append(json_example)
    
    with open(json_path, 'w') as f:
        json.dump(json_data, f, indent=2)
    
    print(f"Saved {split_name} data with {len(examples_data)} examples (sampled every {interval} examples)")
    print(f"Output files: {excel_path}, {csv_path}, and {json_path}")

Resolving data files:   0%|          | 0/98 [00:00<?, ?it/s]

Added example 0 (sample 1/10)
Added example 50 (sample 2/10)
Added example 100 (sample 3/10)
Added example 150 (sample 4/10)
Added example 200 (sample 5/10)
Added example 250 (sample 6/10)
Added example 300 (sample 7/10)
Added example 350 (sample 8/10)
Added example 400 (sample 9/10)
Added example 450 (sample 10/10)
Saved train data with 10 examples (sampled every 50 examples)
Output files: ../raw\train_data.xlsx, ../raw\train_data.csv, and ../raw\train_data.json
